### Definição de Constantes e Parâmetros

Nesta seção, definiremos as propriedades e dimensão dos elementos estruturais.
- Módulo de Elasticidade: Mede a rigidez do material. É definido como a razão entre a tensão aplicada e deformação sofrida pelo material. $$[\frac{Tensão}{Deforma\c{c}ão}] = N/m^2 = Pa$$
- Área de Seção Transversal: Área da seção transversal dos elementos, definida como $$Comprimento_{externo}^2 - Comprimento_{Interno}^2,$$ pois todos os elementos possuem como seção transversal quadrados ocos. $$[Área \ da \ se\c{c}ão \ transversal] = m^2$$
- Comprimento: Distância entre dois nós da estrutura. $$[Comprimento] = m$$
- Ângulo: Orientação dos elementos em relação ao sistema de coordenadas globais. $$[Ângulo] = \frac{m}{m} = rad$$

In [1]:
from sympy import *
from IPython.display import Markdown as md, display
init_printing(num_columns=100)


# Definição de Constantes e Parâmetros
E = Symbol('E')
A = Symbol('A')
L1 = Symbol('L1')
L2 = Symbol('L2')
ang1 = Symbol('ang1')
ang2 = Symbol('ang2')
# Módulo de elasticidade (Pa)
E = 70e9

# Área da seção transversal (m²)
A = 0.012**2-0.009**2

# Comprimentos dos elementos (m)
L1 = sqrt(5)  # Comprimento do elemento 1
L2 = 2           # Comprimento do elemento 2

# Ângulos dos elementos (radianos)
ang1 = atan(-0.5)  # Ângulo do elemento 1
ang2 = 0                # Ângulo do elemento 2

### Cálculo das Matrizes de Rigidez

$$
  \frac{EA}{L}
  \begin{bmatrix}
    \cos^2{\alpha}            & \cos{\alpha}\sin{\alpha}  & -\cos^2{\alpha}           & -\cos{\alpha}\sin{\alpha} \\
    \cos{\alpha}\sin{\alpha}  & \sin^2{\alpha}            & -\cos{\alpha}\sin{\alpha} & -\sin^2{\alpha}           \\
    -\cos^2{\alpha}           & -\cos{\alpha}\sin{\alpha} & \cos^2{\alpha}            & \cos{\alpha}\sin{\alpha}  \\
    -\cos{\alpha}\sin{\alpha} & -\sin^2{\alpha}            & \cos{\alpha}\sin{\alpha}  & \sin^2{\alpha}           \\
  \end{bmatrix}
$$
Essa é a matriz de rigidez das treliças 2D no Sistema Global de Coordenadas

In [2]:
# Constantes de rigidez axial dos elementos
EAL1 = E * A / L1  # Rigidez axial do elemento 1
EAL2 = E * A / L2  # Rigidez axial do elemento 2
# Matriz de Rigidez
def calcular_matriz_rigidez(E, A, L, angulo):
    EA_L = E * A / L
    c = cos(angulo)
    s = sin(angulo)
    return EA_L * Matrix([
        [c**2, c*s, -c**2, -c*s],
        [c*s, s**2, -c*s, -s**2],
        [-c**2, -c*s, c**2, c*s],
        [-c*s, -s**2, c*s, s**2]
    ])

# Matriz de rigidez do elemento 1
k1 = calcular_matriz_rigidez(E, A, L1, ang1)

# Matriz de rigidez do elemento 2
k2 = calcular_matriz_rigidez(E, A, L2, ang2)


print("Matriz de rigidez do elemento 1 (k1):")
display(md(f"${latex(k1)}$"))

print("\nMatriz de rigidez do elemento 2 (k2):")
display(md(f"${latex(k2)}$"))

Matriz de rigidez do elemento 1 (k1):


$\left[\begin{matrix}705600.0 \sqrt{5} & - 352800.0 \sqrt{5} & - 705600.0 \sqrt{5} & 352800.0 \sqrt{5}\\- 352800.0 \sqrt{5} & 176400.0 \sqrt{5} & 352800.0 \sqrt{5} & - 176400.0 \sqrt{5}\\- 705600.0 \sqrt{5} & 352800.0 \sqrt{5} & 705600.0 \sqrt{5} & - 352800.0 \sqrt{5}\\352800.0 \sqrt{5} & - 176400.0 \sqrt{5} & - 352800.0 \sqrt{5} & 176400.0 \sqrt{5}\end{matrix}\right]$


Matriz de rigidez do elemento 2 (k2):


$\left[\begin{matrix}2205000.0 & 0 & -2205000.0 & 0\\0 & 0 & 0 & 0\\-2205000.0 & 0 & 2205000.0 & 0\\0 & 0 & 0 & 0\end{matrix}\right]$

### Montagem da Matriz Global

$$\mathbf{K}_{global}(i,j) = \sum_{e=1}^{n_{elementos}} \mathbf(a,b)$$

In [3]:
def montar_matriz_global(k1, k2):
    # Inicialização da matriz global de rigidez (6x6) com zeros
    matriz_global = Matrix.zeros(6, 6)
    # Inserção da matriz de rigidez do elemento 1 na matriz global
    matriz_global[0:4, 0:4] += k1
    # Inserção da matriz de rigidez do elemento 2 na matriz global
    matriz_global[2:6, 2:6] += k2
    
    return matriz_global

matriz_global = montar_matriz_global(k1, k2)

print("Matriz global de rigidez:")
display(md(f"${latex(matriz_global)}$"))

Matriz global de rigidez:


$\left[\begin{matrix}705600.0 \sqrt{5} & - 352800.0 \sqrt{5} & - 705600.0 \sqrt{5} & 352800.0 \sqrt{5} & 0 & 0\\- 352800.0 \sqrt{5} & 176400.0 \sqrt{5} & 352800.0 \sqrt{5} & - 176400.0 \sqrt{5} & 0 & 0\\- 705600.0 \sqrt{5} & 352800.0 \sqrt{5} & 705600.0 \sqrt{5} + 2205000.0 & - 352800.0 \sqrt{5} & -2205000.0 & 0\\352800.0 \sqrt{5} & - 176400.0 \sqrt{5} & - 352800.0 \sqrt{5} & 176400.0 \sqrt{5} & 0 & 0\\0 & 0 & -2205000.0 & 0 & 2205000.0 & 0\\0 & 0 & 0 & 0 & 0 & 0\end{matrix}\right]$

### Resolução do Sistema


Vetor de Forças:
$$
F = \begin{bmatrix}
? \\
? \\
0 \\
-1000 \\
? \\
? \\
\end{bmatrix}
$$

No sistema reduzido, temos:
$$ \begin{bmatrix}
k_{23} & k_{24} \\
k_{33} & k_{34}
\end{bmatrix}
\cdot
\begin{bmatrix}
u_1 \\
u_2
\end{bmatrix}
=
\begin{bmatrix}
0 \\
-1000
\end{bmatrix}
$$

$$
\Rightarrow
\begin{bmatrix}
u_1 \\
u_2
\end{bmatrix}
=
\begin{bmatrix}
k_{23} & k_{24} \\
k_{33} & k_{34}
\end{bmatrix}^{-1}
\cdot
\begin{bmatrix}
0 \\
-1000
\end{bmatrix} $$

In [4]:
# Redução da matriz global para considerar as condições de contorno
vetor_forcas_reduzido = Matrix([0, -1000])  # Força aplicada no nó correspondente

def resolucao_sistema(matriz_global, vetor_forcas_conhecido):
    # Como sabemos que os deslocamentos na parede são nulos, então cancelamos as linhas e colunas correspondentes, ficando assim apenas uma matriz reduzida
    matriz_reduzida = matriz_global[2:4, 2:4]

    # Resolvendo o sistema de equações para os deslocamentos nodais desconhecidos
    deslocamentos = matriz_reduzida.LUsolve(vetor_forcas_conhecido)
    return deslocamentos

deslocamentos = resolucao_sistema(matriz_global, vetor_forcas_reduzido)

# Exibindo os deslocamentos nodais desconhecidos
print("Deslocamentos nodais desconhecidos:")
display(md(f"${latex(N(deslocamentos, 3))}$"))

Deslocamentos nodais desconhecidos:


$\left[\begin{matrix}-0.000907\\-0.00435\end{matrix}\right]$

### Cálculo das Forças de Reação

$$
\begin{bmatrix}
F_{1x} \\
F_{1y} \\
F_{2x} \\
F_{2y} \\
F_{3x} \\
F_{3y}
\end{bmatrix}
=
\begin{bmatrix}
705600.0\sqrt{5} & -352800.0\sqrt{5} & -705600.0\sqrt{5} & 352800.0\sqrt{5} & 0 & 0 \\
-352800.0\sqrt{5} & 176400.0\sqrt{5} & 352800.0\sqrt{5} & -176400.0\sqrt{5} & 0 & 0 \\
-705600.0\sqrt{5} & 352800.0\sqrt{5} & 705600.0\sqrt{5} + 2205000.0 & -352800.0\sqrt{5} & -2205000.0 & 0 \\
352800.0\sqrt{5} & -176400.0\sqrt{5} & -352800.0\sqrt{5} & 176400.0\sqrt{5} & 0 & 0 \\
0 & 0 & -2205000.0 & 0 & 2205000.0 & 0 \\
0 & 0 & 0 & 0 & 0 & 0 
\end{bmatrix}  
\begin{bmatrix}
0 \\
0 \\
-9{,}07 \cdot 10^{-4} \\
-4{,}35 \cdot 10^{-3} \\
0 \\
0
\end{bmatrix}
=
\begin{bmatrix}
-2000.0 \\
-1000.0 \\
0 \\
-1000.0 \\
2000.0 \\
0.0
\end{bmatrix} \ \mathrm{N}
$$

In [5]:
# Reconstrução do vetor de deslocamentos completo, incluindo os deslocamentos conhecidos (zeros nos apoios)
def calcular_deslocamentos_completos(deslocamentos):
  S = Matrix.zeros(6, 1)
  S[2:4, 0] = deslocamentos
  return S

deslocamentos_completos = calcular_deslocamentos_completos(deslocamentos)

def calcular_forcas_reacao(deslocamentos_completos, matriz_global):
# Cálculo das forças de reação nos apoios utilizando a matriz global de rigidez
  return matriz_global * deslocamentos_completos

forcas_reacao = calcular_forcas_reacao(deslocamentos_completos, matriz_global)

# Exibindo as forças de reação nos apoios
print("Forças de reação nos apoios:")
display(md(f"${latex(N(Matrix(forcas_reacao[0:2])))}$")) # Reações no nó 0
display(md(f"${latex(N(Matrix(forcas_reacao[2:4])))}$")) # Reações no nó 1
display(md(f"${latex(N(Matrix(forcas_reacao[4:6])))}$")) # Reações no nó 2


Forças de reação nos apoios:


$\left[\begin{matrix}-2000.0\\1000.0\end{matrix}\right]$

$\left[\begin{matrix}0\\-1000.0\end{matrix}\right]$

$\left[\begin{matrix}2000.0\\0\end{matrix}\right]$

### Transformação para Coordenadas Locais

Matriz de transformação $ T $:

$$
T = \begin{bmatrix}
\cos{\alpha} & \sin{\alpha} & 0 & 0 \\
-\sin{\alpha} & \cos{\alpha} & 0 & 0 \\
0 & 0 & \cos{\alpha} & \sin{\alpha} \\
0 & 0 & -\sin{\alpha} & \cos{\alpha} \\
\end{bmatrix}
$$

Transformação de deslocamentos globais $ u_g $ para locais $ u_l $:

$$
u_l = T \cdot u_g
$$

In [6]:
# Matriz de transformação para o elemento 1
def calcular_matriz_de_rotacao(angulo):
    T = Matrix([
        [cos(angulo), sin(angulo), 0, 0],
        [-sin(angulo), cos(angulo), 0, 0],
        [0, 0, cos(angulo), sin(angulo)],
        [0, 0, -sin(angulo), cos(angulo)]
    ])
    return T
# Matriz de transformação para o elemento 2
T1 = calcular_matriz_de_rotacao(ang1)
T2 = calcular_matriz_de_rotacao(ang2)

# Transformação dos deslocamentos globais para coordenadas locais
deslocamentos_locais_1 = T1 * deslocamentos_completos[0:4, 0]
deslocamentos_locais_2 = T2 * deslocamentos_completos[2:6, 0]

# Exibindo os deslocamentos transformados para coordenadas locais
print("Deslocamentos em coordenadas locais para o elemento 1:")
display(md(f"${latex(N(deslocamentos_locais_1, 3))}$"))

print("\nDeslocamentos em coordenadas locais para o elemento 2:")
display(md(f"${latex(N(deslocamentos_locais_2, 3))}$"))

Deslocamentos em coordenadas locais para o elemento 1:


$\left[\begin{matrix}0\\0\\0.00113\\-0.0043\end{matrix}\right]$


Deslocamentos em coordenadas locais para o elemento 2:


$\left[\begin{matrix}-0.000907\\-0.00435\\0\\0\end{matrix}\right]$

### Cálculo das Forças Axiais

A força axial é dada por:

$$
F_{axial} = \frac{EA}{L} \cdot 
\begin{bmatrix}
1 & 0 & -1 & 0 \\
0 & 0 & 0 & 0 \\
-1 & 0 & 1 & 0 \\
0 & 0 & 0 & 0 
\end{bmatrix}
\cdot 
\begin{bmatrix}
u_1' \\
u_2' \\
u_3' \\
u_4'
\end{bmatrix}
$$


In [7]:
# Matriz de forças-deslocamentos local para cálculo das forças axiais
def calcular_forcas_axiais(EAL, deslocamento_local):
    matriz_forcas_deslocamento = Matrix([
        [1, 0, -1, 0],
        [0, 0, 0, 0],
        [-1, 0, 1, 0],
        [0, 0, 0, 0]
    ])
    return EAL * matriz_forcas_deslocamento * deslocamento_local

# Cálculo das forças axiais no elemento 1
forcas_axiais_1 = calcular_forcas_axiais(EAL1, deslocamentos_locais_1)
print("Forças axiais no elemento 1:")
display(md(f"${latex(N(forcas_axiais_1, 7))}$"))


# Cálculo das forças axiais no elemento 2
forcas_axiais_2 = calcular_forcas_axiais(EAL2, deslocamentos_locais_2)
print("\nForças axiais no elemento 2:")
display(md(f"${latex(N(forcas_axiais_2))}$"))

Forças axiais no elemento 1:


$\left[\begin{matrix}-2236.068\\0\\2236.068\\0\end{matrix}\right]$


Forças axiais no elemento 2:


$\left[\begin{matrix}-2000.0\\0\\2000.0\\0\end{matrix}\right]$

### Cálculo das Tensões

A tensão axial é dada por:

$$
\sigma_{axial} = \frac{F_{axial}}{A}
$$

In [8]:
# Cálculo das tensões no elemento 1
tensao_1 = (forcas_axiais_1[2, 0] + forcas_axiais_1[3, 0]) / A
display(md(f"Tensão no elemento 1: ${latex(N(tensao_1, 10))}$ ${latex("Pa")}$"))
# Cálculo das tensões no elemento 2
tensao_2 = (forcas_axiais_2[0, 0] + forcas_axiais_2[1, 0]) / A
display(md(f"Tensão no elemento 2: ${latex(N(tensao_2, 10))}$ ${latex("Pa")}$"))

Tensão no elemento 1: $35493142.5$ $\mathtt{\text{Pa}}$

Tensão no elemento 2: $-31746031.75$ $\mathtt{\text{Pa}}$

### Cálculo da Massa e Volume

O volume $ V $ de um elemento de treliça é dado por:

$$
V = A \cdot L
$$

A massa $ m $ de um elemento de treliça pode ser calculada por:

$$
m = \rho \cdot V
$$

Substituindo a expressão do volume, temos:

$$
m = \rho \cdot A \cdot L
$$

In [9]:
# Densidade do material (kg/m³)
densidade = 2800

# Volume da seção transversal (m³)
volume1 = A * L1 
volume2 = A * L2
display(md(f"O volume da barra 1 é: ${latex(volume1)}$ ${latex("m³")}$"))
display(md(f"O volume da barra 2 é: ${latex(volume2)}$ ${latex("m³")}$"))

massa1 = densidade * volume1
massa2 = densidade * volume2
display(md(f"A massa da barra 1 é: ${latex(massa1)}$ ${latex("kg")}$"))
display(md(f"A massa da barra 2 é: ${latex(massa2)}$ ${latex("kg")}$"))

O volume da barra 1 é: $6.3 \cdot 10^{-5} \sqrt{5}$ $\mathtt{\text{m³}}$

O volume da barra 2 é: $0.000126$ $\mathtt{\text{m³}}$

A massa da barra 1 é: $0.1764 \sqrt{5}$ $\mathtt{\text{kg}}$

A massa da barra 2 é: $0.3528$ $\mathtt{\text{kg}}$

# Otimização
Como não queremos alterar a estrutura do projeto, trabalharemos em cima da váriavel do momento de inércia $(I)$

In [10]:
# Cálculo do momento de Inércia da seção transversal
I = (0.012**4-0.009**4)/12
display(md(f"${latex("I =")}$ ${latex(N(I, 6))}$ ${latex("m⁴")}$"))
# Cálculo da força axial de compressão crítica
Pcr = ((pi**2)*E*I)/(L2**2)
display(md(f"$P_{{cr}} = $ ${latex(N(Pcr, 6))}$ ${latex("N")}$"))

$\mathtt{\text{I =}}$ $1.18125 \cdot 10^{-9}$ $\mathtt{\text{m⁴}}$

$P_{cr} = $ $204.023$ $\mathtt{\text{N}}$

Agora, calcularemos a força axial de compressão crítica com o coeficiente de segurança $1{,}2$

In [11]:
forca_axial_seguranca = abs(forcas_axiais_2[0, 0]*1.2)
display(md(f"${latex(N(forca_axial_seguranca, 7))}$ ${latex("N")}$"))

$2400.0$ $\mathtt{\text{N}}$

A carga crítica de flambagem é dada por:

$$P_{cr} = \frac{\pi^2EI}{L^2}$$
$$2400 = \frac{\pi^2\cdot 70 \cdot 10^9 \cdot I}{2^2}$$

Isolando o $I$, temos:

$$I = \frac{2400 \cdot 2^2}{\pi^2 \cdot 70 \cdot 10^9}$$

In [12]:
I = (forca_axial_seguranca * L2**2) / (pi**2 * E)

display(md(f"$I = {latex(N(I, 7))}$ m⁴"))

$I = 1.389548 \cdot 10^{-8}$ m⁴

Sabendo que momento de Inércia da seção Transversal é dado por:

$$I = \frac{a^4_e - a^4_i}{12} $$

Se mantivermos o $a_e$, seria impossível chegar ao valor mínimo, então reforçarei $a_e$ e manterei $a_i$
$$I = \frac{a^4_e - 0{,}009^4}{12} $$
$$a_e = \sqrt[4]{12 \cdot I + 0{,}009^4}$$
Calculando $a_e$

In [13]:
# Calculando a_e
a_e = (12 * I + 0.009**4)**0.25
display(md(f"$a_e =$ ${latex(N(a_e, 7))}$ ${latex("m")}$"))

$a_e =$ $0.02040346$ $\mathtt{\text{m}}$

In [14]:
# Atualizando a área da seção transversal com o novo a_e
A = (a_e**2-0.009**2)

# Recalculando as constantes de rigidez axial dos elementos
EAL1 = E * A / L1
EAL2 = E * A / L2

# Recalculando as matrizes de rigidez dos elementos
k1 = calcular_matriz_rigidez(E, A, L1, ang1)
k2 = calcular_matriz_rigidez(E, A, L2, ang2)

# Recalculando a matriz global de rigidez
matriz_global = montar_matriz_global(k1, k2)

# Recalculando os deslocamentos nodais desconhecidos
deslocamentos = resolucao_sistema(matriz_global, vetor_forcas_reduzido)

# Recalculando o vetor de deslocamentos completo
deslocamentos_completos = calcular_deslocamentos_completos(deslocamentos)

# Recalculando as forças de reação nos apoios
forcas_reacao = calcular_forcas_reacao(deslocamentos_completos, matriz_global)

# Recalculando os deslocamentos em coordenadas locais
deslocamentos_locais_1 = T1 * deslocamentos_completos[0:4, 0]
deslocamentos_locais_2 = T2 * deslocamentos_completos[2:6, 0]

# Recalculando as forças axiais nos elementos
forcas_axiais_1 = calcular_forcas_axiais(EAL1, Matrix(deslocamentos_locais_1))
forcas_axiais_2 = calcular_forcas_axiais(EAL2, Matrix(deslocamentos_locais_2))

# Recalculando as tensões nos elementos
tensao_1 = (forcas_axiais_1[2] + forcas_axiais_1[3]) / A
tensao_2 = (forcas_axiais_2[0] + forcas_axiais_2[1]) / A
# Recalculando a força axial de compressão crítica
Pcr = ((pi**2)*E*I)/(L2**2)

# Exibindo os resultados recalculados
print("Nova área da seção transversal (A):")
display(md(f"${latex(N(A, 4))}$ ${latex("m²")}$"))
print("Nova matriz de rigidez do elemento 1 (k1):")
display(md(f"${latex(N(k1, 9))}$"))
print("\nNova matriz de rigidez do elemento 2 (k2):")
display(md(f"${latex(N(k2, 9))}$"))
print("\nNova matriz global de rigidez:")
display(md(f"${latex(N(matriz_global, 9))}$"))
print("\nNovos deslocamentos nodais desconhecidos:")
print("\nNovos deslocamentos nodais desconhecidos:")
display(md(f"${latex(N(deslocamentos, 4))}$"))

print("\nNovas forças de reação nos apoios:")
display(md(f"${latex(N(Matrix(forcas_reacao[0:2, 0])))}$")) # Reações no nó 0
display(md(f"${latex(N(Matrix(forcas_reacao[2:4, 0])))}$")) # Reações no nó 1
display(md(f"${latex(N(Matrix(forcas_reacao[4:6, 0])))}$")) # Reações no nó 2
print("\nNovas forças axiais no elemento 1:")
display(md(f"${latex(N(forcas_axiais_1, 7))}$"))
print("\nNovas forças axiais no elemento 2:")
display(md(f"${latex(N(forcas_axiais_2))}$"))
print("\nNova tensão no elemento 1:")
display(md(f"${latex(N(tensao_1, 8))}$ ${latex("Pa")}$"))
print("\nNova tensão no elemento 2:")
display(md(f"${latex(N(tensao_2, 8))}$ ${latex("Pa")}$"))
print("\nNova força axial de compressão crítica:")
display(md(f"${latex(N(Pcr))}$ ${latex("N")}$"))

Nova área da seção transversal (A):


$0.0003353$ $\mathtt{\text{m²}}$

Nova matriz de rigidez do elemento 1 (k1):


$\left[\begin{matrix}8397271.2 & -4198635.6 & -8397271.2 & 4198635.6\\-4198635.6 & 2099317.8 & 4198635.6 & -2099317.8\\-8397271.2 & 4198635.6 & 8397271.2 & -4198635.6\\4198635.6 & -2099317.8 & -4198635.6 & 2099317.8\end{matrix}\right]$


Nova matriz de rigidez do elemento 2 (k2):


...

$6668833.1$ $\mathtt{\text{Pa}}$


Nova tensão no elemento 2:


$-5964785.6$ $\mathtt{\text{Pa}}$


Nova força axial de compressão crítica:


$2400.0$ $\mathtt{\text{N}}$

### Tensão de escoamento


A tensão de escoamento com coeficiente de segurança $2$ é dada por

In [15]:
Sy = tensao_1 * 2
display(md(f"${latex(N(Sy))}$"))


$13337666.1655539$

Como podemos ver, a tensão do elemento 1 está dentro da normas de segurança ($\sigma _y = 70 \cdot 10^6$ Pa)

$$$$

In [16]:
# Densidade do material (kg/m³)
densidade = 2800

# Volume da seção transversal (m³)
volume1 = A * L1 
volume2 = A * L2
display(md(f"O volume da barra 1 é: ${latex(N(volume1, 3))}$ m³"))
display(md(f"O volume da barra 2 é: ${latex(N(volume2, 3))}$ m³"))

massa1 = densidade * volume1
massa2 = densidade * volume2
display(md(f"A massa da barra 1 é: ${latex(N(massa1, 6))}$ kg"))
display(md(f"A massa da barra 2 é: ${latex(N(massa2, 6))}$ kg"))

O volume da barra 1 é: $0.00075$ m³

O volume da barra 2 é: $0.000671$ m³

A massa da barra 1 é: $2.09932$ kg

A massa da barra 2 é: $1.87769$ kg

### Representações gráficas das treliças
- Estrutura:

<img src="img/Estrutura.png">

- Força Axial:

<img src="img/ForçaAxial.png">

- Deformação:

Antes:

<img src="img/Deformação.png">

Depois:

<img src="img/Deformação2.png">

- Tensão Normal:

Antes:

<img src="img/TensãoNormal.png">

Depois:

<img src="img/TensãoNormal2.png">

- Volume:

Antes:

<img src="img/Volume.png">

Depois:

<img src="img/Volume2.png">